# Report

## Queue comparison

CLASS QUEUES (from lecture) vs STOCK MARKET QUEUES (LOB)

CLASS QUEUES (M/M/1, G/G/1, M/M/c, etc.)
- “Service” queues: jobs arrive → wait → get served → depart
- Defined by A/B/c/N/K notation (arrival dist., service dist., servers, capacity)
- FIFO or simple disciplines (FIFO, LIFO, priority, random)
- Key parameters: λ (arrival rate), μ (service rate), ρ (utilization)
- Require a server with a service time distribution
- Usually one queue line (or c identical parallel servers)
- Goals: compute W, Wq, L, Lq, stability (ρ < 1)
- No notion of price; all jobs are homogeneous

STOCK MARKET QUEUES (Bid/Ask LOB)
- Not service queues; orders do not receive “service” but wait for matching or cancellation
- Two sides: Bid (max-price priority) and Ask (min-price priority)
- One FIFO queue per price level; many queues in parallel
- Matching happens when buy price ≥ sell price; no service rate μ
- Strict priority: price priority first, FIFO within the same price level
- Orders may be partially filled, canceled, or modified at any time
- Market orders bypass queues and execute immediately
- System is event-driven, not service-time-driven
- Heterogeneous orders: limit, market, cancel, modify, each with price and quantity

KEY DIFFERENCES
- Class queues are time-driven service models; LOB is an event-driven matching system
- Class queues use service rates μ; LOB has no service times or servers
- Class queues have one main queue; LOB has many price-level queues with priority
- Class queues use FIFO; LOB uses price priority → FIFO
- Class queues rely on steady-state formulas; LOB follows matching rules


### VFV Simulation Rate Analysis

A typical range of event rates for the **Vanguard S&P 500 Index ETF (VFV)**, which trades on the Toronto Stock Exchange (TSX) and is highly liquid, would align with a busy, mid-to-high frequency environment.

To determine a realistic range for your simulation's **Poisson rates ($\lambda$)**, we must analyze VFV's **average daily volume** and **liquidity**.

---

### 1. VFV Key Trading Metrics

| Metric | Typical Value | Context for Simulation |
| :--- | :--- | :--- |
| **Average Daily Volume** | ~250,000 to 420,000 units | Total volume to be generated over 23,400 seconds. |
| **Trading Seconds/Day** | **23,400 seconds** | The standard duration for a trading day (9:30 AM to 4:00 PM ET). |
| **Average Bid-Ask Spread** | Very small, often **1-2 cents** | A minimal spread is a hallmark of high liquidity. |

---

### 2. Realistic Range of Event Rates (per second)

We derive the rate of market-moving events (Market Buy/Sell) from the average daily volume and set the Limit Order/Cancel rates higher, reflecting the high-frequency activity typical of ETFs.

#### Market Order Rates (Taker Flow)

These rates determine the volume of executed trades. The total trade rate per second is calculated as:

$$\text{Total Trades per second} = \frac{\text{Average Daily Volume}}{\text{Trading Seconds per Day}}$$

Using an average volume of 350,000 units, this results in approximately $14.96$ trades/second.

| Event Type | Rate Range ($\lambda$) | Rationale |
| :--- | :--- | :--- |
| **`lam_mkt_buy`** | **7.0 to 9.0** | Half the total trade rate, reflecting taker flow. |
| **`lam_mkt_sell`** | **7.0 to 9.0** | |
| **Total Market Trades** | **14.0 to 18.0 / sec** | Highly active trading volume. |

#### Limit Order and Cancel Rates (Maker Flow)

Limit orders and cancellations are typically much more frequent than executed trades, as market participants constantly adjust their quotes.

| Event Type | Rate Range ($\lambda$) | Rationale |
| :--- | :--- | :--- |
| **`lam_limit_buy`** | **10.0 to 20.0** | Orders are constantly placed to maintain the tight spread. |
| **`lam_limit_sell`** | **10.0 to 20.0** | |
| **`lam_cancel`** | **15.0 to 30.0** | Very high cancellation rates are common in high-liquidity assets. |

---

### 3. Summary of a Typical VFV Rate Range

| Parameter | Recommended Range (Events/sec) |
| :--- | :--- |
| **`lam_limit_buy`** | **10.0 to 20.0** |
| **`lam_limit_sell`** | **10.0 to 20.0** |
| **`lam_mkt_buy`** | **7.0 to 9.0** |
| **`lam_mkt_sell`** | **7.0 to 9.0** |
| **`lam_cancel`** | **15.0 to 30.0** |
| **TOTAL RATE** | **42.0 to 88.0+ events/sec** |

> **Note:** A total rate of **40+ events per second** is computationally intensive. If your simulation runs too slowly, you can scale all these rates down proportionally (e.g., divide all rates by 10) to maintain the realistic *ratio* of limit orders, market orders, and cancels.